In [23]:
%load_ext autoreload
%autoreload 3

import numpy as np
import pandas as pd
import gconcorde as cce
from gconcorde import GraphicalConcorde, GraphicalConcordeCV

np.random.seed(3)

# true precision matrix
Omega = np.zeros((3,3), order="C")
Omega[0, 1] = 2.1
Omega[1, 2] = -2.1
Omega += Omega.T
np.fill_diagonal(Omega, 3)
L = np.linalg.cholesky(Omega)

# true covariance matrix
Sigma = np.linalg.inv(Omega)

# observations
X = np.random.multivariate_normal([0]*3, Sigma, 500)

# sample covariance matrix
S = np.cov(X, rowvar=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
def partial_corr(A):
    std_inv = np.diag(np.sqrt(1/np.diag(A)))
    A_corr = -1 * (std_inv @ A @ std_inv)
    np.fill_diagonal(A_corr, 1)
    
    return A_corr

In [3]:
S.flat[::S.shape[0] + 1] = 0
lam_max = np.max(np.abs(S))
lam_min = 0.01 * lam_max
lams = np.logspace(np.log10(lam_min), np.log10(lam_max), 5)

In [4]:
model = GraphicalConcorde(lam1=lams[0], lam2=0, epstol=1e-7, maxitr=500, penalize_diag=True)
model.fit(X)
Omega_hat = model.precision_.toarray()
Omega_hat

lam_mat = np.where(Omega_hat != 0, 0, 100.0)

In [5]:
Omega_corr = partial_corr(Omega)

In [6]:
lam2s = np.linspace(0.01, 0.5, 10)
for lam2 in lam2s:
    model = GraphicalConcorde(lam1=lam_mat, lam2=lam2, epstol=1e-7, maxitr=500, penalize_diag=True)
    model.fit(X)
    Omega_hat = model.precision_.toarray()
    Omega_corr_hat = partial_corr(Omega_hat)
    print(Omega_corr_hat)
    print(np.linalg.norm(Omega_corr - Omega_corr_hat))
    print(' ')
    print(' ')

[[ 1.         -0.69990882 -0.        ]
 [-0.69990882  1.          0.69950522]
 [-0.          0.69950522  1.        ]]
0.0007115076033537158
 
 
[[ 1.         -0.69771143 -0.        ]
 [-0.69771143  1.          0.69721832]
 [-0.          0.69721832  1.        ]]
0.0050941796063759
 
 
[[ 1.         -0.69589634 -0.        ]
 [-0.69589634  1.          0.69533253]
 [-0.          0.69533253  1.        ]]
0.008789224176101187
 
 
[[ 1.         -0.69415725 -0.        ]
 [-0.69415725  1.          0.69353737]
 [-0.          0.69353737  1.        ]]
0.01232098334436549
 
 
[[ 1.         -0.69240218 -0.        ]
 [-0.69240218  1.          0.69173726]
 [-0.          0.69173726  1.        ]]
0.015874489653082205
 
 
[[ 1.         -0.6906078  -0.        ]
 [-0.6906078   1.          0.68990607]
 [-0.          0.68990607  1.        ]]
0.019498761927703865
 
 
[[ 1.         -0.68877239 -0.        ]
 [-0.68877239  1.          0.68804028]
 [-0.          0.68804028  1.        ]]
0.023198880739107655
 
 
[

In [5]:
# CONCORDe CV

print("omega_hat:\n %s\n" % (str(model_cv.precision_.toarray())))
print("hist: (inner_iter_count, delta_subg, objective)\n %s\n" % (str(model_cv.hist_[-3:])))
print("iterations: %d" % (len(model_cv.hist_)))
print("successive norm diff: %e" % (model_cv.hist_[-1,0]))
print("objective: %e \n" % (model_cv.hist_[-1,1]))
print("cv results:\n %s\n" % pd.DataFrame(model_cv.cv_results_))
print("best lam: %s" % model_cv.best_param_)

omega_hat:
 [[ 2.03380077  1.36279999  0.        ]
 [ 1.36279999  1.8841713  -1.36310986]
 [ 0.         -1.36310986  2.03805204]]

hist: (inner_iter_count, delta_subg, objective)
 [[1.88941057e-05 9.45847264e-01]
 [1.86311386e-05 9.45847253e-01]
 [1.83719341e-05 9.45847241e-01]]

iterations: 500
successive norm diff: 1.837193e-05
objective: 9.458472e-01 

cv results:
          lam         score
0   0.116869   7800.831566
1   0.369572   8565.759128
2   1.168689   8934.402873
3   3.695718  12894.612277
4  11.686885  12894.612277

best lam: 0.11686885357906268
